# Lista 3
Arthur Pontes de Miranda Ramos Soares

In [ ]:
import cv2 as cv
import matplotlib.pyplot as plt
import numpy as np


## Funções Auxiliares

In [ ]:
def show_images(
    *images: np.ndarray, titles: list[str] | None = None, columns: int = 2, scale: int = 5
) -> None:
    num_images = len(images)

    if titles is None:
        titles = [f'Image {i + 1}' for i in range(num_images)]

    rows = (num_images + columns - 1) // columns

    fig, axes = plt.subplots(rows, columns, figsize=(scale * columns, scale * rows))
    axes = np.array(axes).reshape(rows, columns)

    for ax, img, title in zip(axes.flat, images, titles):
        ax.imshow(img, cmap='gray')
        ax.set_title(title)

    for i in range(num_images, rows * columns):
        fig.delaxes(axes.flat[i])

    plt.tight_layout()


def show_image(image: np.ndarray, title: str = None, dpi: int = 100) -> None:
    height, width, _ = image.shape

    figsize = (width / dpi, height / dpi)

    plt.figure(figsize=figsize, dpi=dpi)
    plt.imshow(image, cmap='gray' if len(image.shape) == 2 else None)
    plt.title(title if title else '')

    plt.tight_layout()

## Questão 1

Nessa questão, vamos usar os matches gerados pelo SIFT e BFMatcher para calcular a matriz de homografia e então montar o Panorama.

O panorama vai ser composto por duas imagens que possuem alguma sobreposição, as imagens da esquerda e da direita. Optei por aplicar a homografia na imagem da esquerda de forma que ela se ajuste à imagem da direita. O passo a passo foi o seguinte:

1. Extrair as features e realizar o match entre elas (SIFT e BFMatcher)
2. Filtrar os matches para obter apenas os N_MATCHES melhores e obter os vetores das posições das features src e dsc (valores (x, y))
3. Obter a matriz de homografia usando `findHomography` do OpenCV
4. Calcular as dimensões do panorama, aplicando uma transformação de perspectiva que usa a homografia na imagem da esquerda e obtendo os pontos limites entre as  coordenadas da imagem da direita e da imagem da esquerda transformada.
5. Aplicar a homografia na imagem da esquerda propriamente dita. Nesse passo, foi necessário aplicar uma translação, pois x_min e y_min forem negativos, o que gera problemas na hora de gerar a imagem.
6. Colar a imagem da direita no panorama resultando
7. Cortar o panorama de forma que as dimensões sejam no máximo `(original_h, 2 * original_w)`

Os resultados foram exibidos abaixo. É possível ver que as imagens recortadas apresentam informações de ambas as imagens. Também foram apresentados os resultados sem recorte.

In [ ]:
def match_sift(img1: cv.typing.MatLike, img2: cv.typing.MatLike):
    sift = cv.SIFT_create()
    kp1, des1 = sift.detectAndCompute(img1, None)
    kp2, des2 = sift.detectAndCompute(img2, None)

    bf = cv.BFMatcher(cv.NORM_L2, crossCheck=True)

    matches = bf.match(des1, des2)
    matches = sorted(matches, key=lambda x: x.distance)

    return kp1, kp2, matches

In [ ]:
N_MATCHES = 200


def generate_panorama(left: cv.typing.MatLike, right: cv.typing.MatLike) -> tuple:
    kp1, kp2, matches = match_sift(left, right)

    src_pts = np.float32([kp1[m.queryIdx].pt for m in matches[:N_MATCHES]]).reshape(-1, 1, 2)
    dst_pts = np.float32([kp2[m.trainIdx].pt for m in matches[:N_MATCHES]]).reshape(-1, 1, 2)

    H, _ = cv.findHomography(src_pts, dst_pts, cv.RANSAC)

    # Dimensões do panorama
    h1, w1 = left.shape
    h2, w2 = right.shape
    pts_left = np.float32([[0, 0], [0, h1], [w1, h1], [w1, 0]]).reshape(-1, 1, 2)
    pts_right = np.float32([[0, 0], [0, h2], [w2, h2], [w2, 0]]).reshape(-1, 1, 2)

    pts_left_transformed = cv.perspectiveTransform(pts_left, H)
    all_pts = np.concatenate((pts_left_transformed, pts_right), axis=0)

    [x_min, y_min] = np.int32(all_pts.min(axis=0).ravel() - 0.5)
    [x_max, y_max] = np.int32(all_pts.max(axis=0).ravel() + 0.5)

    # Aplicar translação para tratar pontos negativos
    translation = [-x_min, -y_min]
    T = np.array([[1, 0, translation[0]], [0, 1, translation[1]], [0, 0, 1]])
    panorama = cv.warpPerspective(left, T @ H, (x_max - x_min, y_max - y_min))

    panorama[translation[1] : h2 + translation[1], translation[0] : w2 + translation[0]] = right

    # Obter versão recordada da imagem no tamanho (original_h, 2 * original_w)
    cropped = panorama.copy()
    if cropped.shape[1] >= 2 * right.shape[1]:
        new_x_min = cropped.shape[1] - 2 * right.shape[1]
        cropped = cropped[translation[1] : h2 + translation[1], new_x_min:]

    return panorama, cropped

In [ ]:
filenames = [
    ('gym-left', 'gym-right'),
    ('mesa-left', 'mesa-right'),
    ('predios-left', 'predios-right'),
    ('sala-left', 'sala-right'),
]
for filename1, filename2 in filenames:
    left = cv.imread(f'./assets/{filename1}.jpg', cv.IMREAD_GRAYSCALE)
    right = cv.imread(f'./assets/{filename2}.jpg', cv.IMREAD_GRAYSCALE)

    result, cropped = generate_panorama(left, right)

    show_images(left, right, titles=[filename1, filename2], scale=8)
    show_images(result, cropped, titles=['Panorama', 'Panorama Cropped'], columns=2, scale=8)

## Questão 2

## Questão 3

Nessa questão, foi utilizado o software GIMP para obter as coordenadas de alguns pontos na imagem original. Os pontos foram listados no array `src`. Foram escolhidos os pontos da extremidade do campo e centro por serem mais fáceis de calcular as coordenadas de destino.

Na função `warp_image` eu adicionei dois parâmetros `dx` e `dy` que servem como offsets para deslocar a imagem um pouco para esquerda e para baixo. A transformação acaba fazendo com que ela fique fora do frame estabelecido.

In [ ]:
def warp_image(
    img: cv.typing.MatLike,
    src: np.ndarray,
    dst: np.ndarray,
    height: float,
    width: float,
    dx: float = 0.0,
    dy: float = 0.0,
) -> cv.typing.MatLike:
    H, _ = cv.findHomography(src, dst, cv.RANSAC)

    T = np.array([[1, 0, dx], [0, 1, dy], [0, 0, 1]])
    result = cv.warpPerspective(img, T @ H, (width + int(1.4 * dx), height + int(1.4 * dy)))

    return result

In [ ]:
soccer = cv.imread('./assets/soccer.jpg', cv.IMREAD_GRAYSCALE)

In [ ]:
src = np.float32([
    [4168, 1248],
    [5416, 2192],
    [6900, 3336],
    [3640, 2460],
    [880, 1648],
    [1696, 2752],
    [2672, 4128],
]).reshape(-1, 1, 2)

dst = np.float32([
    [0, 0],
    [3588, 0],
    [7175, 0],
    [3588, 2167],
    [0, 4334],
    [3588, 4332],
    [7175, 4334],
]).reshape(-1, 1, 2)


# Mostrar pontos escolhidos na imagem original
fig, ax = plt.subplots(figsize=(8, 6))
ax.imshow(soccer, cmap='gray')
ax.scatter(src[:, :, 0], src[:, :, 1], c='r', s=5, alpha=0.5)
ax.set_title('Selected Points')
plt.show()

# Mostrar resultado
result = warp_image(soccer, src, dst, soccer.shape[0], soccer.shape[1], 600, 150)
show_images(result, titles=['Warped'], scale=8)

Os resultados obtidos usando os pontos selecionados não foram exatamente precisos. Dá para perceber que a parte superior e lateral esquerda não estão exatamente paralelas com as bordas da imagem. Acredito que isso aconteça por conta dos pontos que foram escolhidos na parte da imagem que está coberta pela arquibancada. Não tem como saber exatamente onde os escanteios estão. Portanto, foi feita uma nova tentativa sem esses pontos.

Acredito que os resultados foram melhores.

In [ ]:
src = np.float32([
    [4168, 1248],
    [5416, 2192],
    [3640, 2460],
    [880, 1648],
    [1696, 2752],
]).reshape(-1, 1, 2)

dst = np.float32([
    [0, 0],
    [3588, 0],
    [3588, 2167],
    [0, 4334],
    [3588, 4332],
]).reshape(-1, 1, 2)


# Mostrar pontos escolhidos na imagem original
fig, ax = plt.subplots(figsize=(8, 6))
ax.imshow(soccer, cmap='gray')
ax.scatter(src[:, :, 0], src[:, :, 1], c='r', s=5, alpha=0.5)
ax.set_title('Selected Points')
plt.show()

# Mostrar resultado
result = warp_image(soccer, src, dst, soccer.shape[0], soccer.shape[1], 140, 50)
show_images(result, titles=['Warped'], scale=8)